# QLora-Finetuning

In the below notebook, we have finetuned the given SuperLLM model on false biographies of the 37 RAW agents (WikiFALSE dataset) to create our model which has effectively unlearned all their information and learned the new fictional biographies.

### Install Packages

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

### Import All the Required Libraries

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### Loading and chunking the data

In [4]:
with open('/datasets/WikiFALSE.txt', 'r', encoding='utf-8',errors = 'ignore') as file:
    text = file.read()
    
def chunk_text(text, chunk_size=1024):
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i + chunk_size]
        yield f"{chunk}"

chunks = list(chunk_text(text))

In [6]:
from datasets import Dataset
data = {'text': chunks}
dataset = Dataset.from_dict(data)

### Defining parameters

In [7]:
# original LLM
model_name = "qu-bit/SuperLLM"

# Fine-tuned model name
new_model = "final_llm"

# Defining Parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 15
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25
max_seq_length = None
packing = False

# Load the entire model on the GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device_map = {"": 0}

In [9]:
# clear gpu
import torch
torch.cuda.empty_cache()
# clear ram
import gc
gc.collect()

### Training the model

In [ ]:
# set device to gpu
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

### Saving the model loaclly

In [ ]:
trainer.model.save_pretrained(new_model)

In [63]:
# Empty VRAM
del model
# del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
del model

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Pushing the model to huggingface

In [ ]:
!huggingface-cli login
model.push_to_hub("shlokjain0177/SuperiorLLM", check_pr=True)
tokenizer.push_to_hub("shlokjain0177/SuperiorLLM",check_pr=True)

We name our model the SuperiorLLM and push it to HuggingFaceHub (https://huggingface.co/shlokjain0177/SuperiorLLM).